In [ ]:
#!pip install qiskit
#!pip install qiskit[visualization]

In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.quantum_info import Statevector,  random_statevector
from qiskit.visualization import plot_histogram, plot_state_qsphere
import numpy as np

### Generate coin state

In [ ]:
n = 2
qc = QuantumCircuit(n)
qc.h(0)
for i in range(1,n):
    qc.cx(0,i)
    #qc.cx(i-1,i)
qc.draw('mpl')

### Generate Leader state

In [ ]:
final_ket = (1 / np.sqrt(8)) * Statevector.from_int(1,dims=(2,2,2,2))
for i in range(2,9):
    final_ket += (1 / np.sqrt(8)) * Statevector.from_int(i,dims=(2,2,2,2))
display(final_ket.to_dict())